# DataFrames.jl 介紹 (四): Categorical Data 及 Missing Data

![](https://juliadata.github.io/DataFrames.jl/stable/assets/logo.png)

DataFrames.jl 官方網站: [https://juliadata.github.io/DataFrames.jl/stable/](https://juliadata.github.io/DataFrames.jl/stable/)

DataFrames.jl GitHub: [https://github.com/JuliaData/DataFrames.jl/blob/master/docs/src/index.md](https://github.com/JuliaData/DataFrames.jl/blob/master/docs/src/index.md)

本範例需要用到的套件: `DataFrames`, `CSV`, `CategoricalArrays`

## 0. 安裝

如果尚未安裝過 DataFrames.jl 的話, 執行 `Pkg.add()` 進行安裝

In [1]:
using Pkg

In [2]:
Pkg.add(PackageSpec(name="DataFrames", version="0.20.2"))

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


如果尚未安裝過 CategoricalArrays.jl 的話, 執行 `Pkg.add()` 進行安裝

In [3]:
Pkg.add(PackageSpec(name="CategoricalArrays", version="0.7.7"))

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
  [324d7699] ↑ CategoricalArrays v0.7.1 ⇒ v0.7.7
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [324d7699] ↑ CategoricalArrays v0.7.1 ⇒ v0.7.7


安裝的版本

In [4]:
Pkg.installed()["CategoricalArrays"]

v"0.7.7"

## 1. 建立 CategoricalArray

建立 CategoricalArray 的方式有兩種:
- 透過建構子 `CategoricalArray()`
- 透過 `categorical()` 函式

In [5]:
using CategoricalArrays

┌ Info: Recompiling stale cache file /Users/yellowcc/.julia/compiled/v1.2/CategoricalArrays/RHXoP.ji for CategoricalArrays [324d7699-5711-5eae-9e2f-1d82baa6b597]
└ @ Base loading.jl:1240


### 透過建構子 `CategoricalArray()` 來建立

In [6]:
cv = CategoricalArray(["B", "A", missing, "B", "B"])

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "B"    
 "A"    
 missing
 "B"    
 "B"    

### 透過 `categorical()` 函式

In [7]:
cv2 = categorical(["B", "A", missing, "B", "B"], true, ordered=true)

5-element CategoricalArray{Union{Missing, String},1,UInt8}:
 "B"    
 "A"    
 missing
 "B"    
 "B"    

### `levels()` 與 `levels!()`

呼叫 `levels()` 產生 CategoricalArray 中可出現的類別. 以 cv 的例子來說, A 和 B 是可出現的類別.

In [8]:
levels(cv)

2-element Array{String,1}:
 "A"
 "B"

`levels!()` 函式可以改變取出的順序, 執行過後取出 cv 中類別的順序已改變.

In [9]:
levels!(cv, ["B", "A"])

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "B"    
 "A"    
 missing
 "B"    
 "B"    

In [10]:
levels(cv)

2-element Array{String,1}:
 "B"
 "A"

In [11]:
unique(cv)

3-element Array{Union{Missing, String},1}:
 "B"    
 "A"    
 missing

### `compress()`

`compress()` 函式可以壓縮 CategoricalArray 以節省記憶體的用量.

In [12]:
cv = CategoricalArray(["B", "A", missing, "B", "B"])

# 呼叫 Base.summarysize() 函式查看物件的記憶體使用量
println("原始大小: ", Base.summarysize(cv), " bytes")

原始大小: 818 bytes


In [13]:
cv = compress(cv)
println("壓縮後: ", Base.summarysize(cv), " bytes")

壓縮後: 749 bytes


如前面範例, 呼叫 `categorical()` 函式時, 設定 compress 變數為 true 時, 就可以在建立 CategoricalArray 時同時進行壓縮.

In [14]:
cv2 = categorical(["B", "A", missing, "B", "B"], true)
Base.summarysize(cv2)

749

## 2. 將 DataFrame 中的字串 (String) 欄位轉換為 CategoricalString

在 DataFrame 中如果需要將定欄位轉換為類別型態的話, 可以使用 `categorical!()` 函式.

首先先建立一個 DataFrame, 包含 Int64, String, String 資料型態的 3 個欄位.

In [15]:
using DataFrames

df = DataFrame(index = [1, 2, 3, 4, 5],
               name = ["James", "Jim", "Adam", "Dawn", "Jason"],
               industry = ["製造業", "營建工程業", "批發及零售業", "資通訊服務業", "金融及保險業"])

┌ Info: Recompiling stale cache file /Users/yellowcc/.julia/compiled/v1.2/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1240


5×3 DataFrame
│ Row │ index │ name   │ industry     │
│     │ Int64 │ String │ String       │
├─────┼───────┼────────┼──────────────┤
│ 1   │ 1     │ James  │ 製造業       │
│ 2   │ 2     │ Jim    │ 營建工程業   │
│ 3   │ 3     │ Adam   │ 批發及零售業 │
│ 4   │ 4     │ Dawn   │ 資通訊服務業 │
│ 5   │ 5     │ Jason  │ 金融及保險業 │

In [16]:
# 查看欄位中的元素的資料型態
eltype.(eachcol(df))

3-element Array{DataType,1}:
 Int64 
 String
 String

呼叫 `categorical!()` 函式, 將 industry 欄位轉換為 CategoricalString 型態.

In [17]:
categorical!(df, :industry, compress=true)

5×3 DataFrame
│ Row │ index │ name   │ industry     │
│     │ Int64 │ String │ Categorical… │
├─────┼───────┼────────┼──────────────┤
│ 1   │ 1     │ James  │ 製造業       │
│ 2   │ 2     │ Jim    │ 營建工程業   │
│ 3   │ 3     │ Adam   │ 批發及零售業 │
│ 4   │ 4     │ Dawn   │ 資通訊服務業 │
│ 5   │ 5     │ Jason  │ 金融及保險業 │

In [18]:
eltype.(eachcol(df))

3-element Array{DataType,1}:
 Int64                   
 String                  
 CategoricalString{UInt8}

## 3. Missing 的處理

Julia 提供 Missing 型別來代表當資料中有缺值的情況發生時, 類比於 SQL 的 NULL、R 語言中的 NA.

In [19]:
# 新增 age 欄位 (column)
df.age = [23, 34, missing, 56, 33]
df

5×4 DataFrame
│ Row │ index │ name   │ industry     │ age     │
│     │ Int64 │ String │ Categorical… │ Int64⍰  │
├─────┼───────┼────────┼──────────────┼─────────┤
│ 1   │ 1     │ James  │ 製造業       │ 23      │
│ 2   │ 2     │ Jim    │ 營建工程業   │ 34      │
│ 3   │ 3     │ Adam   │ 批發及零售業 │ missing │
│ 4   │ 4     │ Dawn   │ 資通訊服務業 │ 56      │
│ 5   │ 5     │ Jason  │ 金融及保險業 │ 33      │

In [20]:
x = df[!, 4]

5-element Array{Union{Missing, Int64},1}:
 23       
 34       
   missing
 56       
 33       

### `skipmissing`

使用 `Base.skipmissing()` 函式, 將 vector 中的 missing 值去掉. 呼叫後可以看到回傳的是 Int64 與 missing 聯集的 vector.

In [21]:
skipmissing(x)

Base.SkipMissing{Array{Union{Missing, Int64},1}}(Union{Missing, Int64}[23, 34, missing, 56, 33])

取出非 missing 值.

In [22]:
collect(skipmissing(x))

4-element Array{Int64,1}:
 23
 34
 56
 33

以上面 DataFrame 為例, 希望計算年齡的平均值, 但是需要去掉 missing 值才能計算出正確的平均值.

In [23]:
using Statistics

mean(skipmissing(x))

36.5

### `coalesce()`

配合 `skipmissing()`, 呼叫 `Base.coalesce()` 取代 missing, 下面範例是將 vector 中的 missing 值以年齡的平均值取代.

In [24]:
coalesce.(x, mean(skipmissing(x)))

5-element Array{Real,1}:
 23  
 34  
 36.5
 56  
 33  

### `dropmissing()` 與 `dropmissing!()`

在 DataFrames 中提供 `dropmissing()` 與 `dropmissing!()`, 可將 DataFrame 中含有 missing 的 row 去掉. 兩者不同點在於, `dropmissing!()` 會 in-place 改變 DataFrame.

以上面 DataFrame 為例, 呼叫 `dropmissing()` 後僅剩下一個 row.

In [25]:
dropmissing(df)

4×4 DataFrame
│ Row │ index │ name   │ industry     │ age   │
│     │ Int64 │ String │ Categorical… │ Int64 │
├─────┼───────┼────────┼──────────────┼───────┤
│ 1   │ 1     │ James  │ 製造業       │ 23    │
│ 2   │ 2     │ Jim    │ 營建工程業   │ 34    │
│ 3   │ 4     │ Dawn   │ 資通訊服務業 │ 56    │
│ 4   │ 5     │ Jason  │ 金融及保險業 │ 33    │

可針對特定欄位 (column) 進行處理.

In [26]:
dropmissing(df, :name)

5×4 DataFrame
│ Row │ index │ name   │ industry     │ age     │
│     │ Int64 │ String │ Categorical… │ Int64⍰  │
├─────┼───────┼────────┼──────────────┼─────────┤
│ 1   │ 1     │ James  │ 製造業       │ 23      │
│ 2   │ 2     │ Jim    │ 營建工程業   │ 34      │
│ 3   │ 3     │ Adam   │ 批發及零售業 │ missing │
│ 4   │ 4     │ Dawn   │ 資通訊服務業 │ 56      │
│ 5   │ 5     │ Jason  │ 金融及保險業 │ 33      │